In [1]:
import sys, re
from molvs import Standardizer
from rdkit import Chem
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import errorcode
import operator
from rdkit.Chem.SaltRemover import SaltRemover
import sys
import re
from rdkit import Chem
from rdkit.Chem.Draw import rdMolDraw2D
from optparse import OptionParser
import pandas as pd
import numpy as np
from rdkit.Chem.Descriptors import MolWt, HeavyAtomCount,MolLogP, RingCount, NumAromaticRings, NumHDonors, NumHAcceptors
from rdkit.Chem.Descriptors import NumAromaticRings, qed
from rdkit.Chem.QED import QEDproperties
from rdkit.Chem import rdMMPA
from indexing_call_as_function import cansmirk,add_context_to_smirks,index_from_smiles_re,gen_queries,gen_queries_for_H_transform,add_context_to_lhs
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from molvs import standardize_smiles, Standardizer
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.SaltRemover import SaltRemover

import numpy as np
import pandas as pd
# import seaborn as sn
# import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, BoundaryNorm
from sklearn import neighbors
import matplotlib.patches as mpatches
# import graphviz
from sklearn.tree import export_graphviz
import matplotlib.patches as mpatches

# Download the data and prepare

In [2]:
df_t_half = pd.read_csv('../MMP_files/Chembl_t_half.csv')
df_cl = pd.read_csv('../MMP_files/Clearance.csv')
df_bio = pd.read_csv('../MMP_files/Bioavail_All_Chembl.csv')
df_vdss = pd.read_csv('../MMP_files/Vdss_All_Chembl.csv')
df_herg = pd.read_csv('../MMP_files/Chembl_herg.csv')

In [3]:
df_0 = pd.read_csv('../MMP_files/Chembl_t_half.csv')
df_1 = pd.read_csv('../MMP_files/Clearance.csv')
df_2 = pd.read_csv('../MMP_files/Bioavail_All_Chembl.csv')
df_3 = pd.read_csv('../MMP_files/Vdss_All_Chembl.csv')
df_4 = pd.read_csv('../MMP_files/Chembl_herg.csv')

In [4]:
df_0.dropna(subset = ['canonical_smiles'], inplace=True)
df_1.dropna(subset = ['canonical_smiles'], inplace=True)
df_2.dropna(subset = ['canonical_smiles'], inplace=True)
df_3.dropna(subset = ['canonical_smiles'], inplace=True)
df_4.dropna(subset = ['canonical_smiles'], inplace=True)

In [5]:
df_0['MW'] = df_0['canonical_smiles'].apply(lambda x: MolWt(Chem.MolFromSmiles(x)))
df_1['MW'] = df_1['canonical_smiles'].apply(lambda x: MolWt(Chem.MolFromSmiles(x)))
df_2['MW'] = df_2['canonical_smiles'].apply(lambda x: MolWt(Chem.MolFromSmiles(x)))
df_3['MW'] = df_3['canonical_smiles'].apply(lambda x: MolWt(Chem.MolFromSmiles(x)))
df_4['MW'] = df_4['canonical_smiles'].apply(lambda x: MolWt(Chem.MolFromSmiles(x)))

In [6]:
df_0.shape[0],df_1.shape[0],df_2.shape[0],df_3.shape[0],df_4.shape[0],

(1434, 35476, 5260, 13009, 10154)

In [7]:
df_0 = df_0[df_0['MW']<=600]
df_1 = df_1[df_1['MW']<=600]
df_2 = df_2[df_2['MW']<=600]
df_3 = df_3[df_3['MW']<=600]
df_4 = df_4[df_4['MW']<=600]

In [8]:
df_0.shape[0],df_1.shape[0],df_2.shape[0],df_3.shape[0],df_4.shape[0]

(1158, 32184, 4574, 11807, 9788)

In [9]:
# cols = ['Chembl_half_life',
#         'Chembl_clearance',
#         'Chembl_bioavailability',
#         'Chembl_vdss',
#         'Chembl_hERG']

In [10]:
files = ['../MMP_files/Clearance.csv',
        '../MMP_files/Chembl_t_half.csv',
        '../MMP_files/Bioavail_All_Chembl.csv',
        '../MMP_files/Chembl_herg.csv',
        '../MMP_files/Vdss_All_Chembl.csv',
        ]

In [11]:
cols = ['Chembl_clearance',
        'Chembl_half_life',
        'Chembl_bioavailability',
        'Chembl_hERG',
        'Chembl_vdss'
        ]

In [12]:
# df_t_half = df_t_half[df_t_half['standard_type']=='T1/2']

In [13]:
len(set([x for x in df_cl['molecule_chembl_id']]))

21315

In [14]:
df_cl[df_cl['standard_type']!='CL/F'].shape

(35061, 11)

# Upload chembl compounds into the database

In [15]:
config = {  'user': 'sandeep',
  'password': 'sql',
  'host': 'localhost',
  'database': 'mmpdb',
  'raise_on_warnings': True,
}

link = mysql.connector.connect(**config)

In [16]:
cnx= mysql.connector.connect(**config)
cur = cnx.cursor()

In [17]:
# i = 0
# for i in range(len(cols)):
    
# #     print(cols[i])
    
#     df_ = pd.read_csv(files[i])
    
#     sql = 'DROP TABLE IF EXISTS {} CASCADE;'.format(cols[i])
#     cur.execute(sql)

In [18]:
# cur.execute('DROP TABLE IF EXISTS ikenacomps')

In [19]:
# 
df = pd.concat([df_0,df_1,df_2,df_3,df_4])

df.dropna(subset = ['canonical_smiles'], inplace=True)

In [20]:
id_s = []
for _, x in df.iterrows():
    
    if (x['molecule_chembl_id'] not in id_s):        
        sql = 'INSERT INTO ikenacomps (Molecule_id, CXCSmiles ) ' \
            ' VALUES (%s, %s)'
        cur.execute(sql,(x['molecule_chembl_id'],x['canonical_smiles']
                        ))
        id_s.append(x['molecule_chembl_id'])

In [21]:
def calc_aggregation(group,name):
    name_mean = name+"_mean";
    name_count = name+"_count";
    name_std = name+"_std";
    name_data = name+"_data"
#     print([re.sub('[><= ]+ ','',str(x)) for x in group[name]])
    out = [float(re.sub('[><= ]+ ','',str(x))) for x in group[name]]
    group['mean'] = "{:0.2f}".format(np.mean(out))
    group['count'] = np.size(out)
    if(np.size(out) >1):     
        group['std_dev']= "{:0.2f}".format(np.std(out,ddof=1))
    else:
        group['std_dev']= 0.0
    group['data'] = ";".join([str(x) for x in group[name]])
    return group

# Create tables for Chembl ADMET and Herg

In [22]:
i = 0
for i in range(len(cols)):
    
#     print(cols[i])
    
    df_ = pd.read_csv(files[i])
    
#     sql = 'DROP TABLE IF EXISTS {} CASCADE;'.format(cols[i])
#     cur.execute(sql)

    sql = 'CREATE TABLE if not exists  {} (id integer not null auto_increment, Molecule_id varchar(128) unique, '\
         '{} double, mean double, ' \
         'std_dev varchar(20), ' \
         'count varchar(200), data text, ' \
         'mol_id integer, '\
         'standard_type varchar(20), ' \
         'standard_units varchar(20), ' \
         'assay_chembl_id varchar(40), ' \
         'assay_description text, ' \
         'constraint foreign key (mol_id) '\
         'references ikenacomps(id) on delete cascade on update cascade, '\
         'primary key(id)) ENGINE=InnoDB'.format(cols[i],cols[i])
    cur.execute(sql)
    
    df1 = df_[['molecule_chembl_id','canonical_smiles','standard_value','standard_units','standard_type','assay_chembl_id',
                'assay_description']]
    df1.dropna(subset=['standard_value','standard_units'],inplace=True)
    df1 = pd.DataFrame(df1.groupby("molecule_chembl_id").apply(calc_aggregation, 'standard_value'))
    
    ids = []

    for _,x in df1.iterrows():
        if(x['molecule_chembl_id'] not in ids):
            sql = 'insert into {}  (Molecule_id, {},mean,std_dev,count, data,standard_type, '\
                      'standard_units,assay_chembl_id,assay_description ) ' \
                      'values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'.format(cols[i],cols[i])

            cur.execute(sql,(x['molecule_chembl_id'],x['standard_value'],x['mean'],x['std_dev'],x['count'],x['data']
                            , x['standard_type'], x['standard_units'],x['assay_chembl_id'],x['assay_description']))

            ids.append(x['molecule_chembl_id'])
            
    sql = 'update {} set mol_id = (select ikenacomps.id from ikenacomps ' \
          'where ikenacomps.Molecule_id = {}.Molecule_id)'.format(cols[i],cols[i])
    
    cur.execute(sql,())
    
    
    

C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\envs\my-rdkit-env\lib

# MMP analysis

In [23]:
import time

In [24]:
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

In [25]:
def get_context_type(context):

    mol = Chem.MolFromSmiles(context)
    mol = mol_with_atom_index(mol)
    
    atomindex = []
    atoms = {}
    neigh = {}
    lv1_keys = []
    
    for atom in mol.GetAtoms():
        atoms[atom.GetIdx()] = atom
        neigh[atom.GetIdx()] = atom.GetNeighbors()

        if(atom.GetMass()==0):
            atomindex.append(atom.GetIdx())
            lv1_keys.append(atom.GetIdx())
    
    str_ = []
    for keys in lv1_keys:
        for values in neigh[keys]:
            if(values.GetIsAromatic()):
                str_.append("a")
            else:
                str_.append("A")
                
    return "".join(str_)

In [26]:
# sql = 'DROP TABLE IF EXISTS mmp_chembl CASCADE;'
# cur.execute(sql)
sql = 'create table if not exists mmp_chembl (id integer not null auto_increment, ref_id varchar(50), id_a integer, '\
      'query_id varchar(50), id_b integer, transform varchar(250), transform_id integer, ' \
      'lhs varchar(250), lhs_id integer, context_t varchar(10), context_t_id integer, ' \
      'context varchar(250), context_id integer, primary key(id)) ENGINE=InnoDB'
print(sql)
cur.execute(sql)

create table if not exists mmp_chembl (id integer not null auto_increment, ref_id varchar(50), id_a integer, query_id varchar(50), id_b integer, transform varchar(250), transform_id integer, lhs varchar(250), lhs_id integer, context_t varchar(10), context_t_id integer, context varchar(250), context_id integer, primary key(id)) ENGINE=InnoDB


In [27]:
cur.execute("SELECT transform from transform")
transforms = cur.fetchall()
transforms = [x[0] for x in transforms]

cur.execute("SELECT transform_left from transform_left")
transform_left = cur.fetchall()
one_side = [x[0] for x in transform_left]

cur.execute("SELECT context from context")
context = cur.fetchall()
context_cont = [x[0] for x in context]

cur.execute("SELECT context_type from context_type")
context_type = cur.fetchall()
context_t = [x[0] for x in context_type]


In [28]:
print(len(transforms), len(one_side), len(context_cont), len(context_t))

65486 3702 2946 14


In [29]:
i = 0
for i in range(len(cols)):
    
    df = pd.read_csv(files[i])
    df.dropna(subset = ['canonical_smiles'], inplace=True)
    df = df.drop_duplicates(subset=['molecule_chembl_id'])
    df['MW'] = df['canonical_smiles'].apply(lambda x: MolWt(Chem.MolFromSmiles(x)))
    df = df[df['MW']<=500]
    print("Number of compounds = {}".format(df.shape[0]))
#     if(i==0):
#         df = df[df['standard_type']!='CL/F']
#     df = df[:500]
    print(files[i])
    s= Standardizer()
    remover = SaltRemover(defnData="[Cl,Br,Na,K,F,I,Mg,Ca]")
    df['smiles'] = df['canonical_smiles'].apply(lambda x: Chem.MolToSmiles(s.standardize(remover.StripMol(Chem.MolFromSmiles(x)))))
    df['HeavyAtomCount'] = df['smiles'].apply(lambda x: HeavyAtomCount(Chem.MolFromSmiles(x)))
    df.drop_duplicates(subset=['smiles'],inplace=True)
    print("Starting Fragmentation")
    df['frags'] = df['smiles'].apply(lambda x: rdMMPA.FragmentMol(Chem.MolFromSmiles(x), pattern="[#6+0;!$(*=,#[!#6])]!@!=!#[*]", resultsAsMols=False))
    print("Starting MMP")
    index = index_from_smiles_re(df)
    
    print("Making MMP table")
    start_time = time.time()

    transform_lev0 = []
    context_type = []
    id_left = []
    id_right = []
    contextcont = []
    
    for key in index:
        total = len(index[key])
        if(total == 1):  # Meaning that this context does not have any Mathed Pairs therefore exit the for loop
            continue
        for xa in range(total-1):
            for xb in range(xa+1,total):
                id_a,core_a = index[key][xa].split(";t")
                id_b,core_b = index[key][xb].split(";t")
                if(id_a != id_b):
                    id_a = int(id_a)
                    id_b = int(id_b)
                    if(core_a != core_b):
                        if(df.loc[id_a]['assay_chembl_id'] == df.loc[id_b]['assay_chembl_id']):
                            smirks,context = cansmirk(core_a,core_b,key)
                            id_a = int(id_a)
                            id_b = int(id_b)
                            context_type.append(get_context_type(context))
                            transform_lev0.append(smirks)
                            contextcont.append(context)
                            id_left.append(df.loc[id_a]['molecule_chembl_id'])
                            id_right.append(df.loc[id_b]['molecule_chembl_id'])
                            
    

    end_time = time.time()
    print("total time taken this loop: ", end_time - start_time)
    
    print("Making MMP table 1. Dataframe finished")
    
    df_mmp_lev0 = pd.DataFrame({'Reference_id' :id_left,
                           'Query_id' :id_right,
                           'transform' : transform_lev0,
                           'Ref_Context' :context_type,
                           'Query_Context' :context_type,
                           'Context': contextcont}
                          )

    for _,x in df_mmp_lev0.iterrows():

        trans_a,trans_b = x['transform'].split(">>")
        opposite_transform = trans_b+">>"+trans_a

        if(x['transform'] not in transforms):
            sql = 'INSERT INTO transform (transform) VALUES (%s);'
            cur.execute(sql, (x['transform'], ))
            transforms.append(x['transform'])

        if(opposite_transform not in transforms):
            sql = 'insert into transform (transform) values (%s);'
            cur.execute(sql,(opposite_transform, ))
            transforms.append(opposite_transform)

        if(trans_a not in one_side):
            sql = 'insert into transform_left (transform_left) values (%s);'
            cur.execute(sql,(trans_a, ))
            one_side.append(trans_a)

        if(trans_b not in one_side):
            sql = 'insert into transform_left (transform_left) values (%s);'
            cur.execute(sql,(trans_b, ))
            one_side.append(trans_b)

        if(x['Context'] not in context_cont):
            sql = 'insert into context (context) values (%s);'
            cur.execute(sql,(x['Context'], ))
            context_cont.append(x['Context'])  

        if(x['Ref_Context'] not in context_t):
            sql = 'insert into context_type (context_type) values (%s);'
            cur.execute(sql,(x['Ref_Context'], ))
            context_t.append(x['Ref_Context']) 
            
            

        sql = 'insert into mmp_chembl (ref_id,query_id,transform, lhs, context_t, context)' \
              'values (%s,%s,%s,%s,%s,%s);'
        cur.execute(sql,(x['Reference_id'],x['Query_id'], x['transform'], trans_a, x['Ref_Context'], x['Context']))


        sql = 'insert into mmp_chembl (ref_id,query_id,transform, lhs, context_t, context)' \
              'values (%s,%s,%s,%s,%s,%s);'
        cur.execute(sql,(x['Query_id'],x['Reference_id'], opposite_transform, trans_b, x['Query_Context'], x['Context']))
        
    
    print("Updating MMP table 2. Dataframe finished")
    
    sql = 'update mmp_chembl set id_a = (select ikenacomps.id from ikenacomps ' \
          'where ikenacomps.Molecule_id = mmp_chembl.ref_id)'
    cur.execute(sql,())

    sql = 'update mmp_chembl set id_b = (select ikenacomps.id from ikenacomps ' \
          'where ikenacomps.Molecule_id = mmp_chembl.query_id)'
    cur.execute(sql,())


    sql = 'update mmp_chembl set transform_id = (select transform.id from transform ' \
          'where transform.transform = mmp_chembl.transform)'
    cur.execute(sql,())


    sql = 'update mmp_chembl set lhs_id = (select transform_left.id from transform_left ' \
          'where transform_left.transform_left = mmp_chembl.lhs)'
    cur.execute(sql,())


    sql = 'update mmp_chembl set context_t_id = (select context_type.id from context_type ' \
          'where context_type.context_type = mmp_chembl.context_t)'
    cur.execute(sql,())


    sql = 'update mmp_chembl set context_id = (select context.id from context ' \
          'where context.context = mmp_chembl.context)'
    cur.execute(sql,())
        
    print("Making MMP table 3. Dataframe finished")
    
    
    
    

Number of compounds = 15210
../MMP_files/Clearance.csv
Starting Fragmentation
Starting MMP
Making MMP table
total time taken this loop:  124.58048820495605
Making MMP table 1. Dataframe finished
Updating MMP table 2. Dataframe finished
Making MMP table 3. Dataframe finished
Number of compounds = 694
../MMP_files/Chembl_t_half.csv
Starting Fragmentation
Starting MMP
Making MMP table
total time taken this loop:  41.74084401130676
Making MMP table 1. Dataframe finished
Updating MMP table 2. Dataframe finished
Making MMP table 3. Dataframe finished
Number of compounds = 2422
../MMP_files/Bioavail_All_Chembl.csv
Starting Fragmentation
Starting MMP
Making MMP table
total time taken this loop:  17.027918815612793
Making MMP table 1. Dataframe finished
Updating MMP table 2. Dataframe finished
Making MMP table 3. Dataframe finished
Number of compounds = 6576
../MMP_files/Chembl_herg.csv
Starting Fragmentation
Starting MMP
Making MMP table
total time taken this loop:  94.54447746276855
Making MM

In [30]:
sql = 'alter table mmp_chembl drop column ref_id,' \
      'drop query_id, drop transform, drop lhs, drop context_t, drop context, drop context_t_id '
cur.execute(sql,())  

In [31]:
cnx.commit()
cnx.close()